In [1]:
import os
import sys
from tqdm import tqdm
from PIL import Image
import numpy as np

sys.path.extend(['..'])

from utils.config import process_config

import tensorflow as tf
from tensorflow.layers import (conv2d, max_pooling2d, average_pooling2d, batch_normalization, dropout, dense)
from tensorflow.nn import (relu, sigmoid, softmax, leaky_relu)

from sklearn.utils import shuffle

In [2]:
DATA = '../data/data_clean/'
CONF = '../configs/roman.json'

In [3]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

def shuffle_sim(a, b):
    assert a.shape[0] == a.shape[0], 'Shapes must be equal'
    
    ind = np.arange(a.shape[0])
    np.random.shuffle(ind)
    return a[ind], b[ind]

In [4]:
def read_train_test(path_to_data):
    data = {}
    for dset in ['train', 'test']:
        path_ = os.path.join(path_to_data, dset)
        X, Y = [], []
        classes = [d for d in os.listdir(path_) if os.path.isdir(os.path.join(path_, d))]
        classes.sort()
        
        for cl in classes:
            y = np.zeros((1, 8), dtype=np.int32)
            y[0, int(cl) - 1] = 1
            
            cl_path = os.path.join(path_, cl)
            filenames = [os.path.join(cl_path, pict) for pict in os.listdir(cl_path) if pict.endswith('.jpg')]
            
            for im in filenames:
                image = np.asarray(Image.open(im), dtype=np.float32)
                X.append(normalize(image).reshape((1, image.shape[0], image.shape[1], image.shape[2])))
                Y.append(y)
        
        a, b = shuffle_sim(np.concatenate(X), np.concatenate(Y))
        data[dset] = ([a, b])
    return data

In [11]:
class Model():
    
    def __init__(self, config, sess_cf, learning_rate):
        self.lr = learning_rate
        self.sess = tf.Session(config=sess_cf)

        self.x = tf.placeholder(dtype=tf.float32, shape=(None, config.image_size, config.image_size, 3))
        self.y = tf.placeholder(dtype=tf.int32, shape=(None, 8))
        self.training = tf.placeholder(dtype=tf.bool, shape=())

        global_step = tf.Variable(1, name='global_step', trainable=False, dtype=tf.int32)
        self.step = tf.assign(global_step, global_step + 1)
        
        self.model()
        
        self.summ_writer = tf.summary.FileWriter(config.summary_dir, graph=self.sess.graph)
        self.sess.run(tf.global_variables_initializer())
    
    def block(self, inp,
              ch,
              num,
              c_ker=[(3, 3), (3, 3)],
              c_str=[(1, 1), (1, 1)],
              act=relu,
              mp_ker=(2, 2),
              mp_str=(2, 2),
              mode='conc'):
    
        with tf.variable_scope('block_' + str(num), reuse=tf.AUTO_REUSE):
            conv = conv2d(inp, ch, c_ker[0], strides=c_str[0])
            bn = batch_normalization(conv)
            out = act(bn)
            out = dropout(out, 0.2)
            tf.summary.histogram('conv1', conv)
            print(out.shape)
            
            conv = conv2d(out, ch, c_ker[1], strides=c_str[1])
            bn = batch_normalization(conv)
            out = act(bn)
            tf.summary.histogram('conv2', conv)
            print(out.shape)
            
            mp = max_pooling2d(out, mp_ker, strides=mp_str)
            if mode == 'mp':
                out = mp
            elif mode == 'conc':
                out = tf.concat([mp, average_pooling2d(out, mp_ker, mp_str)], -1)
            else:
                raise ValueError('Unknown mode.')
            
            print(out.shape)
        return out
    
    def model(self):
        with tf.name_scope('layers'):
            out = self.block(self.x, 16, 1, mode='conc')
            out = self.block(out, 32, 2, mode='conc')
            out = self.block(out, 64, 3, mode='conc')
            out = self.block(out, 256, 4, c_str=[(1, 1), (2, 2)], mode='mp')
            
            dim = np.prod(out.shape[1:])
            out = tf.reshape(out, [-1, dim])
            print(out.shape)
            
            dense_l = dense(out, 128)
            tf.summary.histogram('dense2', dense_l)
            out = batch_normalization(dense_l)
            out = leaky_relu(out, alpha=0.01)
            out = dropout(out, rate=0.7, training=self.training)
            print(out.shape)

            self.predictions = dense(out, 8, activation=softmax)
            tf.summary.histogram('pred', self.predictions)

        with tf.name_scope('metrics'):    
            amax_labels = tf.argmax(self.y, 1)
            amax_pred   = tf.argmax(self.predictions, 1)

            self.loss = tf.losses.softmax_cross_entropy(self.y, self.predictions)        
            self.acc = tf.reduce_mean(tf.cast(tf.equal(amax_labels, amax_pred), dtype=tf.float32))

            self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.loss)

            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.acc)

        self.summary = tf.summary.merge_all()
        
    def train(self, dat, dat_v, epochs):
        for epoch in range(epochs):
            loss, acc, _, summary, step = self.sess.run([
                self.loss, self.acc, self.optimizer, self.summary, self.step
            ],
                                                feed_dict={
                                                    self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: True
                                                })

            self.summ_writer.add_summary(summary, step)
            print('EP: {:3d}\tLOSS: {:.10f}\tACC: {:.10f}'.format(
                epoch, loss, acc))

            if epoch % 10 == 0 and epoch != 0:
                self.test(dat_v)
                
    def test(self, dat):
        loss, acc = self.sess.run([self.loss, self.acc],
                                         feed_dict={self.x: dat[0],
                                                    self.y: dat[1],
                                                    self.training: False})

        print('\tVALIDATION\tLOSS: {:.10f}\tACC: {:.10f}'.format(loss, acc))
    
    def close(self):
        self.sess.close()
        tf.reset_default_graph()
        

In [12]:
m.close()

In [13]:
config_tf = tf.ConfigProto(allow_soft_placement=True)
config_tf.gpu_options.allow_growth = True
config_tf.gpu_options.per_process_gpu_memory_fraction = 0.95

In [14]:
config = process_config(CONF)
config['exp_name'] = '4b_ap_mp'
config['summary_dir'] = '../experiments/' + config['exp_name'] + '/summary'

In [15]:
m = Model(config, config_tf, 5e-4)

(?, 126, 126, 16)
(?, 124, 124, 16)
(?, 62, 62, 32)
(?, 60, 60, 32)
(?, 58, 58, 32)
(?, 29, 29, 64)
(?, 27, 27, 64)
(?, 25, 25, 64)
(?, 12, 12, 128)
(?, 10, 10, 256)
(?, 4, 4, 256)
(?, 2, 2, 256)
(?, 1024)
(?, 128)


In [16]:
dat = read_train_test(DATA)

In [17]:
m.train(dat['train'], dat['test'], 300)

EP:   0	LOSS: 2.0795323849	ACC: 0.1475054175
EP:   1	LOSS: 2.0796723366	ACC: 0.1062906757
EP:   2	LOSS: 2.0793409348	ACC: 0.1279826462
EP:   3	LOSS: 2.0787518024	ACC: 0.1323210448
EP:   4	LOSS: 2.0788757801	ACC: 0.1431670338
EP:   5	LOSS: 2.0786354542	ACC: 0.1409978271
EP:   6	LOSS: 2.0809714794	ACC: 0.1214750558
EP:   7	LOSS: 2.0769274235	ACC: 0.1431670338
EP:   8	LOSS: 2.0793395042	ACC: 0.1301518381
EP:   9	LOSS: 2.0775713921	ACC: 0.1301518381
EP:  10	LOSS: 2.0770213604	ACC: 0.1409978271
	VALIDATION	LOSS: 2.0753703117	ACC: 0.1416666657
EP:  11	LOSS: 2.0734472275	ACC: 0.1778741926
EP:  12	LOSS: 2.0722839832	ACC: 0.1431670338
EP:  13	LOSS: 2.0711917877	ACC: 0.1800433844
EP:  14	LOSS: 2.0759205818	ACC: 0.1301518381
EP:  15	LOSS: 2.0657722950	ACC: 0.1605206132
EP:  16	LOSS: 2.0559904575	ACC: 0.1952277720
EP:  17	LOSS: 2.0527453423	ACC: 0.1930585653
EP:  18	LOSS: 2.0460200310	ACC: 0.1843817830
EP:  19	LOSS: 2.0345833302	ACC: 0.2255965322
EP:  20	LOSS: 2.0139260292	ACC: 0.2299349308
	VALID

EP: 165	LOSS: 1.4446377754	ACC: 0.8394793868
EP: 166	LOSS: 1.4442954063	ACC: 0.8329718113
EP: 167	LOSS: 1.4369292259	ACC: 0.8373101950
EP: 168	LOSS: 1.4355489016	ACC: 0.8438177705
EP: 169	LOSS: 1.4386259317	ACC: 0.8438177705
EP: 170	LOSS: 1.4332518578	ACC: 0.8416485786
	VALIDATION	LOSS: 1.4825416803	ACC: 0.7916666865
EP: 171	LOSS: 1.4266726971	ACC: 0.8568329811
EP: 172	LOSS: 1.4349930286	ACC: 0.8459869623
EP: 173	LOSS: 1.4345687628	ACC: 0.8416485786
EP: 174	LOSS: 1.4323545694	ACC: 0.8416485786
EP: 175	LOSS: 1.4077625275	ACC: 0.8698481321
EP: 176	LOSS: 1.4030251503	ACC: 0.8785249591
EP: 177	LOSS: 1.4351207018	ACC: 0.8329718113
EP: 178	LOSS: 1.4311336279	ACC: 0.8459869623
EP: 179	LOSS: 1.4235153198	ACC: 0.8524945974
EP: 180	LOSS: 1.4258950949	ACC: 0.8524945974
	VALIDATION	LOSS: 1.4968678951	ACC: 0.7749999762
EP: 181	LOSS: 1.4207015038	ACC: 0.8524945974
EP: 182	LOSS: 1.4201316833	ACC: 0.8481561542
EP: 183	LOSS: 1.4175926447	ACC: 0.8568329811
EP: 184	LOSS: 1.4104760885	ACC: 0.8676789403
EP